RAG-LangChain llm

gpu 비전모델에 따로 배정

In [1]:
import os
import requests
import base64
import io
import fitz
from PIL import Image
from typing import Optional, List, Any
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM

# ============================================
# 0️⃣ 환경 설정
# ============================================
OLLAMA_URL = "http://localhost:11434"
LLM_MODEL = "gpt-oss"              # 텍스트 분석용
VISION_MODEL = "llava"             # 이미지 분석용
EMBED_MODEL = "nomic-embed-text"   # 임베딩용
VISION_GPU = "1"                   # Vision 모델용 GPU 번호 (예: GPU1)
pdf_path = "/data1/workspace/pdfs/1.pdf"

# ============================================
# 1️⃣ PDF 텍스트 로드 및 분할
# ============================================
print("[1] PDF 텍스트 로드 중...")
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()

print("[2] 텍스트 분할 중...")
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = splitter.split_documents(docs)

# ============================================
# 2️⃣ 임베딩 및 벡터스토어 생성
# ============================================
print("[3] 임베딩 및 DB 생성 중...")
embeddings = OllamaEmbeddings(model=EMBED_MODEL, base_url=OLLAMA_URL)
vectorstore = FAISS.from_documents(splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

# ============================================
# 3️⃣ Vision 모델로 이미지 약물명 추출
# ============================================
def extract_drugs_from_figures(pdf_path, model=VISION_MODEL, gpu_id=VISION_GPU):
    print(f"[4] Vision 모델({model})로 이미지 약물명 추출 중... (GPU {gpu_id})")
    doc = fitz.open(pdf_path)
    all_drugs = []

    for page_idx, page in enumerate(doc):
        image_list = page.get_images(full=True)
        for img_index, img in enumerate(image_list):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]

            image = Image.open(io.BytesIO(image_bytes))
            image.thumbnail((1024, 1024))  # 처리속도 향상용 축소

            # Base64 인코딩
            buffer = io.BytesIO()
            image.save(buffer, format="JPEG")
            img_b64 = base64.b64encode(buffer.getvalue()).decode()

            # Vision 모델 호출
            payload = {
                "model": model,
                "prompt": (
                    "Extract only the generic drug names visible in this biomedical figure or table. "
                    "Return names separated by semicolons (;). No explanation."
                ),
                "images": [img_b64],
                "stream": False
            }

            env = os.environ.copy()
            env["CUDA_VISIBLE_DEVICES"] = gpu_id  # Vision GPU 고정 👈

            try:
                res = requests.post(f"{OLLAMA_URL}/api/generate", json=payload, timeout=120)
                res.raise_for_status()
                result = res.json().get("response", "").strip()
                if result:
                    all_drugs.append(result)
                    print(f"✅ Page {page_idx+1}-{img_index+1}: {result}")
            except Exception as e:
                print(f"⚠️ Vision 분석 오류 (page {page_idx+1}): {e}")

    doc.close()
    return "; ".join(all_drugs)

# ============================================
# 4️⃣ LLM 정의 (Ollama API)
# ============================================
class OllamaLLM(LLM):
    model_name: str = LLM_MODEL
    base_url: str = OLLAMA_URL
    timeout: int = 300

    def _call(self, prompt: str, stop: Optional[List[str]] = None, run_manager: Optional[Any] = None) -> str:
        payload = {"model": self.model_name, "prompt": prompt, "stream": False}
        try:
            r = requests.post(f"{self.base_url}/api/generate", json=payload, timeout=self.timeout)
            r.raise_for_status()
            return r.json()["response"]
        except Exception as e:
            raise RuntimeError(f"Ollama LLM 오류: {e}")

    @property
    def _llm_type(self) -> str:
        return "ollama"

# ============================================
# 5️⃣ 프롬프트 정의
# ============================================
prompt_template = """You are a biomedical text analysis assistant.

==== Document Context ====
{context}
==== End ====

Task:
- Extract only **generic drug names** mentioned in the Results section.
- If missing, extract from the Methods section.
- Exclude genes, proteins, and references.
- Return names separated by semicolons (;), with no explanations.

Example:
nivolumab; pembrolizumab; sunitinib; fulvestrant

Question: {question}

Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# ============================================
# 6️⃣ RAG 체인 구성
# ============================================
print("[5] RAG 체인 구성 중...")
llm = OllamaLLM()
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True,
    verbose=False
)

# ============================================
# 7️⃣ 실행
# ============================================
print("[6] Vision 분석 + 본문 분석 실행 중...")

vision_drugs = extract_drugs_from_figures(pdf_path)
question = "Extract all generic drug names mentioned in this paper."
response = chain({"query": question})

# 최종 결과 병합
combined = response["result"]
if vision_drugs:
    combined += f"; {vision_drugs}"

# ============================================
# ✅ 결과 출력
# ============================================
print("\n✅ 최종 약물명 추출 결과:")
print(combined)


[1] PDF 텍스트 로드 중...
[2] 텍스트 분할 중...
[3] 임베딩 및 DB 생성 중...
[5] RAG 체인 구성 중...
[6] Vision 분석 + 본문 분석 실행 중...
[4] Vision 모델(llava)로 이미지 약물명 추출 중... (GPU 1)
✅ Page 1-1: 5-Fluorouracil; Tamoxifen; Cyclophosphamide; Methotrexate; Doxorubicin
✅ Page 1-2: IDD
✅ Page 1-3: Elsevier
✅ Page 1-4: Metformin; Glucophage; Gluco-Met
✅ Page 5-1: AKB2; HSP90; BAX; MCL-1; RAS; P53; PTEN; CDH1
✅ Page 6-1: Cetirizine, Dextromethane, Methylprednisolone, Tocilizumab; Enoxaprina, Ibuprofen; Salbutamol, Albuterol; Aspirin, Acetaminophen; Bendamustine; Cisplatin, Cyclophosphamide; Dacarbazine; Doxorubicin; Etomidate; Fluorouracil; Ifosfamide; Oxaliplatin; Pemetrexed; Paclitaxel; Vinorelbine; Vincristine; Bortezomib; Ceritinib; Ibrutinib; Lenalidomide; Sunitinib; Temsirolimus
✅ Page 6-2: There are no visible drug names in this image. The image displays a graph related to clinical trial data, with x-axis labeled as "E," y-axis labeled as "Score_e," and the title reads "Enrollment Score vs Time."
✅ Page 6-3: None of

/tmp/ipykernel_3411421/1458027315.py:164: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain({"query": question})



✅ 최종 약물명 추출 결과:
ciclib; BMS-265246; S63845; ABT-737; VE-822; MK-8776; MG132; sotorasib; adavosertib; puromycin; 5-Fluorouracil; Tamoxifen; Cyclophosphamide; Methotrexate; Doxorubicin; IDD; Elsevier; Metformin; Glucophage; Gluco-Met; AKB2; HSP90; BAX; MCL-1; RAS; P53; PTEN; CDH1; Cetirizine, Dextromethane, Methylprednisolone, Tocilizumab; Enoxaprina, Ibuprofen; Salbutamol, Albuterol; Aspirin, Acetaminophen; Bendamustine; Cisplatin, Cyclophosphamide; Dacarbazine; Doxorubicin; Etomidate; Fluorouracil; Ifosfamide; Oxaliplatin; Pemetrexed; Paclitaxel; Vinorelbine; Vincristine; Bortezomib; Ceritinib; Ibrutinib; Lenalidomide; Sunitinib; Temsirolimus; There are no visible drug names in this image. The image displays a graph related to clinical trial data, with x-axis labeled as "E," y-axis labeled as "Score_e," and the title reads "Enrollment Score vs Time."; None of the image provided contains any generic drug names. It is a graph related to medical data, but not directly showing drug names.